In [1]:
# Loading the marvel graph
marvel_graph = sc.textFile('/FileStore/tables/Marvel_graph-bca37.txt')

In [2]:
marvel_graph = marvel_graph.map(lambda x: x.split(' '))

In [3]:
# setting the source and target value as broadcast variables(immutable variables)
# in this way they will be shared among all the nodes in cluster
source = sc.broadcast('5306')
target = sc.broadcast('14')

In [4]:
# setting the counter value to 0 which will be
# shared among all the nodes in the cluster
counter = sc.accumulator(0)

In [5]:
# converting it  to rdd of tuple (node, connections)
marvel_connection = marvel_graph.map(lambda x: (x[0],x[1:-1]))

In [6]:
# total number of nodes in the graph
marvel_connection.count()

Out[ 107 ]: 6589

In [7]:
marvel_connection.take(1)

Out[ 108 ]: 
[('5988',
 ['748',
 '1722',
 '3752',
 '4655',
 '5743',
 '1872',
 '3413',
 '5527',
 '6368',
 '6085',
 '4319',
 '4728',
 '1636',
 '2397',
 '3364',
 '4001',
 '1614',
 '1819',
 '1585',
 '732',
 '2660',
 '3952',
 '2507',
 '3891',
 '2070',
 '2239',
 '2602',
 '612',
 '1352',
 '5447',
 '4548',
 '1596',
 '5488',
 '1605',
 '5517',
 '11',
 '479',
 '2554',
 '2043',
 '17',
 '865',
 '4292',
 '6312',
 '473',
 '534',
 '1479',
 '6375',
 '4456'])]

In [8]:
# combining the node having same key as one single node
marvel_connection = marvel_connection.reduceByKey(lambda x,y: x+y)

In [9]:
# total number of distinct nodes which have some connection
# is  equal to 6486
marvel_connection.count()

Out[ 110 ]: 6486

In [10]:
'''
this function basically creates rdd of tuple of form (nodeId, (connections, is_explored_or_not, distance))
is_explored_or_not basically has 3 vaues:
1. READY -> means this node is ready to be explored in subsequent iterations
2. NOT READY -> means this node is not ready to be explored as its immediate parents are not yet explored
3. DONE -> means this node has been completely explored and it should not be expllored again to prevent infinite loops

Distance -> means how much distance is the node away from the source.
Therefore, for source, distance = 0
and for other nodes we initialize the distance with large number(9999) initially
'''
def createNode(line):
  node = line[0]
  connection = line[1]
  explored = 'NOT READY'
  distance = 9999
  if node == source.value:
    distance = 0
    explored = 'READY'
  return (node, (connection, explored, distance))

In [11]:
marvel_nodes = marvel_connection.map(lambda x: createNode(x))

In [12]:
marvel_nodes.filter(lambda x:x[0]==source.value).collect()

Out[ 114 ]: 
[('5306',
 (['1718',
 '4024',
 '5981',
 '5986',
 '5984',
 '340',
 '3998',
 '3995',
 '3994',
 '349',
 '3990',
 '6356',
 '967',
 '4933',
 '960',
 '4733',
 '4731',
 '2315',
 '5855',
 '5858',
 '293',
 '291',
 '3771',
 '3770',
 '3777',
 '3775',
 '968',
 '3779',
 '4522',
 '270',
 '272',
 '273',
 '274',
 '4650',
 '278',
 '279',
 '4658',
 '4660',
 '1780',
 '2262',
 '2266',
 '2446',
 '2449',
 '107',
 '2046',
 '2045',
 '2040',
 '2041',
 '2048',
 '4145',
 '6372',
 '3519',
 '3517',
 '3515',
 '6206',
 '6207',
 '2689',
 '6202',
 '6203',
 '2686',
 '6208',
 '99',
 '98',
 '91',
 '1623',
 '5183',
 '3205',
 '1992',
 '1994',
 '1628',
 '3209',
 '3208',
 '2865',
 '555',
 '557',
 '5480',
 '5458',
 '3341',
 '5455',
 '1197',
 '1196',
 '1756',
 '1750',
 '1177',
 '1175',
 '1173',
 '4253',
 '4252',
 '5533',
 '5532',
 '5537',
 '5535',
 '1005',
 '1287',
 '1286',
 '1289',
 '689',
 '3431',
 '685',
 '1572',
 '1571',
 '686',
 '458',
 '620',
 '4383',
 '4381',
 '5622',
 '4384',
 '1370',
 '405',
 '403',
 '1376',
 '3834',
 '3835',
 '3836',
 '3837',
 '3832',
 '4033',
 '5996',
 '5999',
 '377',
 '392',
 '6141',
 '6140',
 '6144',
 '399',
 '398',
 '2309',
 '2303',
 '4726',
 '4721',
 '4722',
 '5846',
 '5843',
 '5842',
 '5841',
 '5840',
 '5849',
 '3748',
 '3740',
 '240',
 '4629',
 '2275',
 '2273',
 '2279',
 '2459',
 '2450',
 '2453',
 '179',
 '178',
 '2052',
 '2055',
 '2054',
 '2057',
 '2056',
 '3690',
 '3691',
 '4530',
 '3696',
 '6367',
 '6366',
 '6365',
 '3524',
 '3523',
 '3520',
 '3086',
 '6217',
 '6216',
 '6214',
 '6211',
 '2698',
 '4651',
 '2693',
 '2692',
 '6219',
 '6218',
 '3233',
 '3231',
 '5199',
 '5198',
 '1616',
 '1966',
 '5196',
 '1613',
 '5193',
 '4553',
 '4550',
 '3354',
 '3357',
 '3356',
 '3353',
 '3359',
 '5440',
 '5446',
 '1766',
 '1765',
 '1142',
 '1146',
 '1147',
 '5508',
 '5506',
 '5504',
 '5505',
 '693',
 '1544',
 '1545',
 '1543',
 '542',
 '543',
 '547',
 '544',
 '545',
 '6403',
 '1787',
 '5616',
 '5615',
 '411',
 '412',
 '1385',
 '1380',
 '1381',
 '1383',
 '3827',
 '3822',
 '3829',
 '3828',
 '5135',
 '4283',
 '4286',
 '4289',
 '361',
 '381',
 '6178',
 '387',
 '388',
 '4758',
 '4759',
 '4751',
 '4752',
 '4754',
 '4757',
 '1076',
 '5874',
 '5875',
 '5878',
 '3751',
 '3750',
 '3754',
 '3757',
 '3756',
 '2193',
 '2194',
 '2199',
 '2203',
 '2209',
 '2422',
 '2423',
 '2424',
 '900',
 '2426',
 '4810',
 '169',
 '164',
 '906',
 '167',
 '163',
 '4839',
 '6353',
 '4529',
 '6351',
 '4526',
 '3686',
 '6196',
 '6190',
 '6278',
 '2511',
 '2731',
 '2733',
 '5296',
 '5297',
 '5294',
 '5295',
 '5290',
 '3530',
 '3532',
 '3538',
 '5057',
 '5051',
 '5053',
 '1814',
 '1817',
 '1818',
 '6227',
 '6225',
 '6228',
 '6229',
 '3229',
 '3228',
 '1979',
 '3225',
 '1976',
 '1602',
 '3220',
 '3223',
 '1970',
 '808',
 '908',
 '801',
 '800',
 '807',
 '2847',
 '2842',
 '2841',
 '3320',
 '5477',
 '5471',
 '3329',
 '1777',
 '1776',
 '5270',
 '5271',
 '1773',
 '1779',
 '5279',
 '1159',
 '1155',
 '1156',
 '1152',
 '5512',
 '5516',
 '3035',
 '3036',
 '3037',
 '2994',
 '2995',
 '3032',
 '533',
 '57',
 '2993',
 '3030',
 '3031',
 '1904',
 '5131',
 '1397',
 '1396',
 '422',
 '425',
 '424',
 '3595',
 '3810',
 '3818',
 '6427',
 '6420',
 '6423',
 '4297',
 '4290',
 '4293',
 '6168',
 '6161',
 '6160',
 '6163',
 '6162',
 '6165',
 '6164',
 '6166',
 '4749',
 '4748',
 '4741',
 '4747',
 '4746',
 '4745',
 '5868',
 '5867',
 '5863',
 '4715',
 '2181',
 '229',
 '2184',
 '4607',
 '4603',
 '4600',
 '2212',
 '2680',
 '2219',
 '2218',
 '152',
 '155',
 '154',
 '2430',
 '2436',
 '2435',
 '6341',
 '4518',
 '4519',
 '4513',
 '4511',
 '5763',
 '6185',
 '6184',
 '6183',
 '4204',
 '2509',
 '2506',
 '2504',
 '2503',
 '2502',
 '2726',
 '5281',
 '5280',
 '5284',
 '2729',
 '2728',
 '5047',
 '5046',
 '5043',
 '1807',
 '3780',
 '3781',
 '3786',
 '3789',
 '4997',
 '6238',
 '4991',
 '6235',
 '6230',
 '3258',
 '3259',
 '3250',
 '1940',
 '3254',
 '1947',
 '1944',
 '819',
 '815',
 '816',
 '1490',
 '5463',
 '5465',
 '1496',
 '420',
 '1499',
 '1968',
 '5262',
 '5269',
 '1129',
 '2093',
 '1121',
 '4577',
 '1125',
 '1127'

In [13]:
counter.value = 0

In [14]:
'''
assigning the rdd marvel_node to bfstraversal rdd
just for better understanding
'''
bfsTraversal = marvel_nodes

In [15]:
'''
This is a mapper function which explores a node
and adds its adjacent connection into READY state
and updates their distance, it also changes state of
currecnt node to DONE after it is completely explored
'''
def checkForConnections(node):
  node_id = node[0]
  data = node[1]
  connections = data[0]
  explored = data[1]
  distance = data[2]
  result = list()
  if explored == 'READY':
    for connection in connections:
      if connection == target.value:
        counter.add(1)
      newNodeId = connection
      dist = distance + 1
      explored = 'READY'
      result.append((newNodeId,(list(), explored, dist)))
    result.append((node_id,(list(), 'DONE', distance)))
  else:
    result.append((node_id,(connections, explored, distance)))
  return result

In [16]:
'''
This reducer function reduces the bfsTraversal rdd
inorder to have latest status(READY, DONE or NOT READY)
and latest minimum from the source node
'''
def reducer(node1, node2):
  explored1 = node1[1]
  explored2 = node2[1]
  if explored1 == 'DONE' or explored2 == 'DONE':
    return (list(), 'DONE', min(node1[2], node2[2]))
  elif explored1 == 'READY' or explored2 == 'READY':
    return (node1[0] if len(node1[0])>len(node2[0]) else node2[0], 'READY', min(node1[2], node2[2]))
  else:
    pass

In [17]:
'''
Taking an uppper bound of 20 iteration
i.e the target would not be farther than
20 hops from the source node

After calling the mapper function we
check the value of acccumulator counter
if it is more than 0, it means that
target node has been found in iteration = i+1
and we break from the loop
'''
for i in range(20):
  bfsTraversal = bfsTraversal.flatMap(checkForConnections)
  print(bfsTraversal.count())
  bfsTraversal.collect()
  if counter.value > 0:
    print("target {} found at a distancce of {} from source {}".format(target.value, i+1, source.value))
    break
  bfsTraversal = bfsTraversal.reduceByKey(reducer)

8227
220615
target 14 found at a distancce of 2 from source 5306